In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt
import sklearn as sk
import sklearn.metrics as metrics
import os

In [2]:
pd.options.mode.use_inf_as_na = True
DATA_FOLDER = '/home/roger/code/kaggle/predict-future-sales/data'

transactions    = pd.read_csv(os.path.join(DATA_FOLDER, 'sales_train.csv'))
items           = pd.read_csv(os.path.join(DATA_FOLDER, 'items.csv'))
item_categories = pd.read_csv(os.path.join(DATA_FOLDER, 'item_categories_en.csv'))
shops           = pd.read_csv(os.path.join(DATA_FOLDER, 'shops_en.csv'))
test            = pd.read_csv(os.path.join(DATA_FOLDER, 'test.csv'))
sample          = pd.read_csv(os.path.join(DATA_FOLDER, 'sample_submission.csv'))

In [3]:
print(transactions.head(2).T)
print(items.head(2).T)
print(item_categories.head(2).T)
print(shops.head(2).T)

                         0           1
date            02.01.2013  03.01.2013
date_block_num           0           0
shop_id                 59          25
item_id              22154        2552
item_price           999.0       899.0
item_cnt_day           1.0         1.0
                                                          0  \
item_name         ! ВО ВЛАСТИ НАВАЖДЕНИЯ (ПЛАСТ.)         D   
item_id                                                   0   
item_category_id                                         40   

                                                                  1  
item_name         !ABBYY FineReader 12 Professional Edition Full...  
item_id                                                           1  
item_category_id                                                 76  
                                             0                  1
item_category_name  PC - Headsets / Headphones  Accessories - PS2
item_category_id                             0                

In [4]:
LAST_TRAIN_MONTH=transactions["date_block_num"].max()

In [5]:
def agg_by_month(transactions):
    ds = (transactions[["date_block_num", "shop_id", "item_id", "item_cnt_day"]]
     .groupby(["date_block_num", "shop_id", "item_id"])
     .agg({"item_cnt_day":"sum"})
     .rename(columns={"item_cnt_day":"item_cnt_month"})
     .reset_index()
    )
    return ds

def split_train_val(ds, split_month=LAST_TRAIN_MONTH):
    '''
    Split dataset into 
        train (date_block_num < month)
        test (date_block_num >= month)
    '''
    
    train_ds = ds[ds["date_block_num"] < split_month]
    val_ds = ds[ds["date_block_num"] == split_month]
    
    return train_ds, val_ds

def val_score(predictions, val_ds):
    '''
    Calculates the root mean squared error between prediction and validation
    '''
    temp = val_ds.rename(columns={"item_cnt_month":"actual_sales"}).merge(
        predictions.rename(columns={"item_cnt_month":"predicted_sales"}), 
        on=["shop_id", "item_id"], 
        how="outer")
    temp["predicted_sales"] = temp["predicted_sales"].fillna(0).clip(0, 20)
    temp["actual_sales"] = temp["actual_sales"].fillna(0).clip(0, 20)
    return np.sqrt(metrics.mean_squared_error(temp["actual_sales"], temp["predicted_sales"]))

## Prev month baseline

In [6]:
monthly_sales = agg_by_month(transactions)
train_ds, val_ds = split_train_val(monthly_sales)

In [7]:
predictions = (train_ds[train_ds["date_block_num"]==train_ds["date_block_num"].max()]
               .drop(columns="date_block_num"))
predictions["item_cnt_month"] = predictions["item_cnt_month"].clip(0, 20)

In [8]:
val_score(predictions, val_ds)

2.5909703492454925

In [9]:
baseline = test.merge(
    val_ds.drop(columns="date_block_num"), 
    on=["shop_id", "item_id"],
    how="left")
baseline["item_cnt_month"] = baseline["item_cnt_month"].fillna(0).clip(0, 20)
baseline[["ID", "item_cnt_month"]].to_csv("comp_submissions/month33_clipped.csv", index=False)
# 1.16777

## Basic tree based baselines

In [35]:
import catboost as cb
from catboost.utils import get_gpu_device_count

In [11]:
monthly_sales = agg_by_month(transactions)

In [12]:
train, val = split_train_val(monthly_sales)

train_x = train[["shop_id", "item_id"]]
train_y = train["item_cnt_month"]
val_x = val[["shop_id", "item_id"]]
val_y = val["item_cnt_month"]

train_pool = cb.Pool(train_x,
                     train_y,
                     cat_features=["shop_id", "item_id"])
val_pool = cb.Pool(val_x,
                   val_y,
                   cat_features=["shop_id", "item_id"])

In [41]:
model = cb.CatBoostRegressor(learning_rate=1
                            ,loss_function="RMSE"
                            ,iterations=4000)

In [42]:
model.fit(train_pool)

Custom logger is already specified. Specify more than one logger at same time is not thread safe.

0:	learn: 6.3903368	total: 241ms	remaining: 16m 3s
1:	learn: 6.2956187	total: 406ms	remaining: 13m 31s
2:	learn: 6.2274136	total: 633ms	remaining: 14m 3s
3:	learn: 6.1200216	total: 753ms	remaining: 12m 31s
4:	learn: 6.0837361	total: 859ms	remaining: 11m 26s
5:	learn: 6.0689404	total: 969ms	remaining: 10m 45s
6:	learn: 6.0382800	total: 1.07s	remaining: 10m 13s
7:	learn: 6.0316283	total: 1.18s	remaining: 9m 49s
8:	learn: 6.0222275	total: 1.28s	remaining: 9m 30s
9:	learn: 6.0159994	total: 1.39s	remaining: 9m 13s
10:	learn: 6.0122549	total: 1.49s	remaining: 9m 1s
11:	learn: 5.9969569	total: 1.61s	remaining: 8m 53s
12:	learn: 5.9929048	total: 1.71s	remaining: 8m 44s
13:	learn: 5.9892004	total: 1.82s	remaining: 8m 37s
14:	learn: 5.9804293	total: 1.92s	remaining: 8m 30s
15:	learn: 5.9775595	total: 2.03s	remaining: 8m 24s
16:	learn: 5.9718832	total: 2.12s	remaining: 8m 17s
17:	learn: 5.9683702	total: 2.23s	remaining: 8m 12s
18:	learn: 5.9645805	total: 2.33s	remaining: 8m 7s
19:	learn: 5.961156

159:	learn: 5.7846545	total: 17.4s	remaining: 6m 57s
160:	learn: 5.7843997	total: 17.5s	remaining: 6m 57s
161:	learn: 5.7842422	total: 17.6s	remaining: 6m 57s
162:	learn: 5.7841182	total: 17.7s	remaining: 6m 57s
163:	learn: 5.7841063	total: 17.8s	remaining: 6m 57s
164:	learn: 5.7839595	total: 18s	remaining: 6m 57s
165:	learn: 5.7835959	total: 18.1s	remaining: 6m 57s
166:	learn: 5.7817772	total: 18.2s	remaining: 6m 57s
167:	learn: 5.7816108	total: 18.3s	remaining: 6m 57s
168:	learn: 5.7815170	total: 18.4s	remaining: 6m 56s
169:	learn: 5.7808600	total: 18.5s	remaining: 6m 56s
170:	learn: 5.7807825	total: 18.6s	remaining: 6m 56s
171:	learn: 5.7804857	total: 18.7s	remaining: 6m 56s
172:	learn: 5.7803590	total: 18.8s	remaining: 6m 56s
173:	learn: 5.7799771	total: 18.9s	remaining: 6m 56s
174:	learn: 5.7785140	total: 19s	remaining: 6m 55s
175:	learn: 5.7784831	total: 19.1s	remaining: 6m 55s
176:	learn: 5.7779165	total: 19.3s	remaining: 6m 55s
177:	learn: 5.7768335	total: 19.4s	remaining: 6m 5

316:	learn: 5.7195298	total: 35s	remaining: 6m 46s
317:	learn: 5.7193977	total: 35.1s	remaining: 6m 46s
318:	learn: 5.7190165	total: 35.2s	remaining: 6m 46s
319:	learn: 5.7188665	total: 35.3s	remaining: 6m 46s
320:	learn: 5.7186053	total: 35.4s	remaining: 6m 46s
321:	learn: 5.7184038	total: 35.5s	remaining: 6m 45s
322:	learn: 5.7181631	total: 35.6s	remaining: 6m 45s
323:	learn: 5.7170135	total: 35.8s	remaining: 6m 45s
324:	learn: 5.7168607	total: 35.9s	remaining: 6m 45s
325:	learn: 5.7166772	total: 36s	remaining: 6m 45s
326:	learn: 5.7165239	total: 36.1s	remaining: 6m 45s
327:	learn: 5.7164154	total: 36.2s	remaining: 6m 45s
328:	learn: 5.7151831	total: 36.3s	remaining: 6m 45s
329:	learn: 5.7150618	total: 36.5s	remaining: 6m 45s
330:	learn: 5.7147905	total: 36.6s	remaining: 6m 45s
331:	learn: 5.7145345	total: 36.7s	remaining: 6m 45s
332:	learn: 5.7134680	total: 36.8s	remaining: 6m 45s
333:	learn: 5.7132600	total: 36.9s	remaining: 6m 45s
334:	learn: 5.7130677	total: 37s	remaining: 6m 44s

472:	learn: 5.6873258	total: 52.9s	remaining: 6m 34s
473:	learn: 5.6873179	total: 53s	remaining: 6m 34s
474:	learn: 5.6870934	total: 53.2s	remaining: 6m 34s
475:	learn: 5.6870540	total: 53.3s	remaining: 6m 34s
476:	learn: 5.6867941	total: 53.4s	remaining: 6m 34s
477:	learn: 5.6866888	total: 53.5s	remaining: 6m 34s
478:	learn: 5.6865090	total: 53.6s	remaining: 6m 34s
479:	learn: 5.6864788	total: 53.7s	remaining: 6m 34s
480:	learn: 5.6864190	total: 53.9s	remaining: 6m 33s
481:	learn: 5.6860289	total: 54s	remaining: 6m 33s
482:	learn: 5.6858043	total: 54.1s	remaining: 6m 33s
483:	learn: 5.6855528	total: 54.2s	remaining: 6m 33s
484:	learn: 5.6853166	total: 54.3s	remaining: 6m 33s
485:	learn: 5.6850799	total: 54.4s	remaining: 6m 33s
486:	learn: 5.6849650	total: 54.5s	remaining: 6m 32s
487:	learn: 5.6848861	total: 54.6s	remaining: 6m 32s
488:	learn: 5.6846159	total: 54.7s	remaining: 6m 32s
489:	learn: 5.6845270	total: 54.8s	remaining: 6m 32s
490:	learn: 5.6844374	total: 54.9s	remaining: 6m 3

...

3353:	learn: 5.5275001	total: 6m 10s	remaining: 1m 11s
3354:	learn: 5.5274814	total: 6m 10s	remaining: 1m 11s
3355:	learn: 5.5274628	total: 6m 10s	remaining: 1m 11s
3356:	learn: 5.5274528	total: 6m 10s	remaining: 1m 11s
3357:	learn: 5.5274197	total: 6m 10s	remaining: 1m 10s
3358:	learn: 5.5274007	total: 6m 11s	remaining: 1m 10s
3359:	learn: 5.5273938	total: 6m 11s	remaining: 1m 10s
3360:	learn: 5.5273840	total: 6m 11s	remaining: 1m 10s
3361:	learn: 5.5273750	total: 6m 11s	remaining: 1m 10s
3362:	learn: 5.5273636	total: 6m 11s	remaining: 1m 10s
3363:	learn: 5.5273598	total: 6m 11s	remaining: 1m 10s
3364:	learn: 5.5273556	total: 6m 11s	remaining: 1m 10s
3365:	learn: 5.5273546	total: 6m 11s	remaining: 1m 10s
3366:	learn: 5.5273305	total: 6m 12s	remaining: 1m 9s
3367:	learn: 5.5272971	total: 6m 12s	remaining: 1m 9s
3368:	learn: 5.5272920	total: 6m 12s	remaining: 1m 9s
3369:	learn: 5.5272757	total: 6m 12s	remaining: 1m 9s
3370:	learn: 5.5272710	total: 6m 12s	remaining: 1m 9s
3371:	learn: 5.

3506:	learn: 5.5241310	total: 6m 27s	remaining: 54.5s
3507:	learn: 5.5241201	total: 6m 27s	remaining: 54.4s
3508:	learn: 5.5239832	total: 6m 27s	remaining: 54.2s
3509:	learn: 5.5239793	total: 6m 27s	remaining: 54.1s
3510:	learn: 5.5239632	total: 6m 27s	remaining: 54s
3511:	learn: 5.5239382	total: 6m 28s	remaining: 53.9s
3512:	learn: 5.5239223	total: 6m 28s	remaining: 53.8s
3513:	learn: 5.5239200	total: 6m 28s	remaining: 53.7s
3514:	learn: 5.5239195	total: 6m 28s	remaining: 53.6s
3515:	learn: 5.5238981	total: 6m 28s	remaining: 53.5s
3516:	learn: 5.5238830	total: 6m 28s	remaining: 53.4s
3517:	learn: 5.5238313	total: 6m 28s	remaining: 53.3s
3518:	learn: 5.5237879	total: 6m 28s	remaining: 53.1s
3519:	learn: 5.5237732	total: 6m 28s	remaining: 53s
3520:	learn: 5.5237668	total: 6m 29s	remaining: 52.9s
3521:	learn: 5.5237616	total: 6m 29s	remaining: 52.8s
3522:	learn: 5.5237501	total: 6m 29s	remaining: 52.7s
3523:	learn: 5.5237472	total: 6m 29s	remaining: 52.6s
3524:	learn: 5.5236490	total: 6m

3659:	learn: 5.5211262	total: 6m 44s	remaining: 37.6s
3660:	learn: 5.5210871	total: 6m 44s	remaining: 37.5s
3661:	learn: 5.5210431	total: 6m 44s	remaining: 37.4s
3662:	learn: 5.5210299	total: 6m 44s	remaining: 37.2s
3663:	learn: 5.5210177	total: 6m 44s	remaining: 37.1s
3664:	learn: 5.5209988	total: 6m 45s	remaining: 37s
3665:	learn: 5.5209743	total: 6m 45s	remaining: 36.9s
3666:	learn: 5.5209666	total: 6m 45s	remaining: 36.8s
3667:	learn: 5.5209484	total: 6m 45s	remaining: 36.7s
3668:	learn: 5.5209006	total: 6m 45s	remaining: 36.6s
3669:	learn: 5.5208947	total: 6m 45s	remaining: 36.5s
3670:	learn: 5.5208716	total: 6m 45s	remaining: 36.4s
3671:	learn: 5.5208330	total: 6m 45s	remaining: 36.2s
3672:	learn: 5.5208230	total: 6m 45s	remaining: 36.1s
3673:	learn: 5.5208122	total: 6m 46s	remaining: 36s
3674:	learn: 5.5207903	total: 6m 46s	remaining: 35.9s
3675:	learn: 5.5207696	total: 6m 46s	remaining: 35.8s
3676:	learn: 5.5207544	total: 6m 46s	remaining: 35.7s
3677:	learn: 5.5207126	total: 6m

3813:	learn: 5.5178658	total: 7m 2s	remaining: 20.6s
3814:	learn: 5.5178468	total: 7m 2s	remaining: 20.5s
3815:	learn: 5.5178095	total: 7m 2s	remaining: 20.4s
3816:	learn: 5.5178038	total: 7m 2s	remaining: 20.3s
3817:	learn: 5.5177665	total: 7m 2s	remaining: 20.1s
3818:	learn: 5.5177429	total: 7m 2s	remaining: 20s
3819:	learn: 5.5177388	total: 7m 2s	remaining: 19.9s
3820:	learn: 5.5176077	total: 7m 2s	remaining: 19.8s
3821:	learn: 5.5175838	total: 7m 3s	remaining: 19.7s
3822:	learn: 5.5175813	total: 7m 3s	remaining: 19.6s
3823:	learn: 5.5175526	total: 7m 3s	remaining: 19.5s
3824:	learn: 5.5175249	total: 7m 3s	remaining: 19.4s
3825:	learn: 5.5174772	total: 7m 3s	remaining: 19.3s
3826:	learn: 5.5174602	total: 7m 3s	remaining: 19.1s
3827:	learn: 5.5174333	total: 7m 3s	remaining: 19s
3828:	learn: 5.5174207	total: 7m 3s	remaining: 18.9s
3829:	learn: 5.5174109	total: 7m 3s	remaining: 18.8s
3830:	learn: 5.5174083	total: 7m 4s	remaining: 18.7s
3831:	learn: 5.5174081	total: 7m 4s	remaining: 18.

3967:	learn: 5.5137301	total: 7m 19s	remaining: 3.54s
3968:	learn: 5.5137130	total: 7m 19s	remaining: 3.43s
3969:	learn: 5.5137109	total: 7m 19s	remaining: 3.32s
3970:	learn: 5.5136983	total: 7m 19s	remaining: 3.21s
3971:	learn: 5.5136967	total: 7m 19s	remaining: 3.1s
3972:	learn: 5.5136852	total: 7m 19s	remaining: 2.99s
3973:	learn: 5.5136823	total: 7m 19s	remaining: 2.88s
3974:	learn: 5.5136755	total: 7m 19s	remaining: 2.77s
3975:	learn: 5.5136708	total: 7m 20s	remaining: 2.65s
3976:	learn: 5.5136643	total: 7m 20s	remaining: 2.54s
3977:	learn: 5.5136570	total: 7m 20s	remaining: 2.43s
3978:	learn: 5.5136485	total: 7m 20s	remaining: 2.32s
3979:	learn: 5.5136424	total: 7m 20s	remaining: 2.21s
3980:	learn: 5.5136315	total: 7m 20s	remaining: 2.1s
3981:	learn: 5.5136159	total: 7m 20s	remaining: 1.99s
3982:	learn: 5.5136131	total: 7m 20s	remaining: 1.88s
3983:	learn: 5.5136126	total: 7m 20s	remaining: 1.77s
3984:	learn: 5.5135554	total: 7m 20s	remaining: 1.66s
3985:	learn: 5.5135283	total: 

In [43]:
pred = pd.DataFrame(model.predict(val_pool), columns=["item_cnt_month"])
pred.index = val_x.index

In [44]:
pred = pd.concat([val_x, pred], axis="columns")

In [45]:
print(pred[:5])
print(val_y[:5])
print(val_score(pred,val))
# 3.0967744882888923

         shop_id  item_id  item_cnt_month
1577593        2       31        1.325907
1577594        2      486        2.650179
1577595        2      787        1.372331
1577596        2      794        1.147345
1577597        2      968        1.279213
1577593    1.0
1577594    3.0
1577595    1.0
1577596    1.0
1577597    1.0
Name: item_cnt_month, dtype: float64
3.142115458747038
